In [ ]:
%matplotlib inline
import matplotlib.pylab as plt
import seaborn as sbn; sbn.set()
import os
import pandas as pd
import _pickle as pickle

In [ ]:
test_name = 'params_select'
test_id = 'test01'
test_dir = './' + test_name + '_' + test_id + '/'

df_historys = pd.DataFrame()
df_params = pd.DataFrame()
df_validate = pd.DataFrame()
df_test = pd.DataFrame()
for root, dirs, files in os.walk(test_dir, topdown=False):
    identity = os.path.basename(root)
    for name in files:
        if name == 'history.pkl':
            with open(os.path.join(root, name), 'rb') as f:
                history = pickle.load(f)
                df_historys = df_historys.append({'id': identity, 'history': history}, ignore_index=True)
        if name == 'params.pkl':
            with open(os.path.join(root, name), 'rb') as f:
                params = pickle.load(f)
                params['id'] = identity
                df_params = df_params.append(params, ignore_index=True)
        if name == 'validate.pkl':
            with open(os.path.join(root, name), 'rb') as f:
                validate = pickle.load(f)
                validate['id'] = identity
                df_validate = df_validate.append(validate, ignore_index=True)
        if name == 'test.pkl':
            with open(os.path.join(root, name), 'rb') as f:
                test = pickle.load(f)
                test['id'] = identity
                df_test = df_test.append(test, ignore_index=True)

In [ ]:
new_test_columns = []
for col in df_test.columns.values.tolist():
    if col != 'id': 
        new_test_columns.append('test_' + col)
    else:
        new_test_columns.append(col)
df_test.columns = new_test_columns

In [ ]:
results = []
for result in [df_historys, df_params, df_validate, df_test]:
    result = result.set_index('id')
    results.append(result)
results = pd.concat(results, axis=1)
results = results.sort_values(['sharpe_ratio'], ascending=False)

In [ ]:
results.columns

In [ ]:
i_columns = ['activation', 'batch_size', 'dropout', 'epochs',
       'is_BN_1', 'is_BN_2', 'is_BN_3', 'lr',
       'recurrent_dropout', 'shuffle', 'time_steps', 'units1', 'units2',
       'units3', 'annual_return', 'sharpe_ratio', 'test_annual_return', 'test_sharpe_ratio']

In [ ]:
i_results = results.loc[:, i_columns]

In [ ]:
i_results.loc[:, ['annual_return', 'sharpe_ratio', 'test_annual_return', 'test_sharpe_ratio']].plot(figsize=(21, 7))

In [ ]:
i_results.head()

In [ ]:
top = 20
for i in range(top):
    validate_returns = results.iloc[i]['returns']
    test_returns = results.iloc[i]['test_returns']
    (validate_returns.unstack(level=0).fillna(0).mean(axis=1) + 1).cumprod().plot(figsize=(21, 7))
    (test_returns.unstack(level=0).fillna(0).mean(axis=1) + 1).cumprod().plot(figsize=(21, 7))
    plt.show()

In [ ]:
for i in range(top):
    validate_returns = results.iloc[i]['returns']
    test_returns = results.iloc[i]['test_returns']
    validate_returns_per_stk = (validate_returns.unstack(level=0).fillna(0) + 1).cumprod()
    test_returns_per_stk = (test_returns.unstack(level=0).fillna(0) + 1).cumprod()
    returns_per_stk = pd.concat([validate_returns_per_stk, test_returns_per_stk], axis=1)
    returns_per_stk.plot(figsize=(21, 7))
    plt.show()

In [ ]:
data_set = pickle.load(open(os.path.join(test_dir, 'data.pkl'), 'rb'))

In [ ]:
data_set['test']

In [ ]:
idx_slice = pd.IndexSlice
data_set['test'].loc[idx_slice[('603228.XSHG',)]]

In [ ]:
validate = data_set['validate']
test = data_set['test']
validate = validate.drop(columns=['label'])
test = test.drop(columns=['label'])

In [ ]:
n = 9
validate_Y = results.iloc[n]['Y']
test_Y = results.iloc[n]['test_Y']

In [ ]:
validate1  = pd.concat([validate, validate_Y], axis=1)
test1  = pd.concat([test, test_Y], axis=1)

In [ ]:
validate1.index.get_level_values(level=0).unique()

In [ ]:
idx_slice = pd.IndexSlice

In [ ]:
stk_predict = test1.loc[idx_slice[('000158.XSHE',)]]
stk_predict = stk_predict.reset_index().reset_index()

In [ ]:
fig, ax = plt.subplots(1, figsize=(21, 7))
stk_predict['close'].plot(x='index', y='close', figsize=(21, 7), ax=ax)
stk_predict[stk_predict['label'] == 2].plot.scatter(x='index', y='close', s=20, c='r', figsize=(21, 7), ax=ax)
stk_predict[stk_predict['label'] == 0].plot.scatter(x='index', y='close', s=20, c='g', figsize=(21, 7), ax=ax)